In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from urlAnalysis import create_graph

In [ ]:
G = create_graph()

In [ ]:
pubs = []
for r in G.predecessors('http://sci-lens.org#repository'):
    for n in G.predecessors(r):
        pubs.append([n, G.in_degree(n)])
pubs = pd.DataFrame(pubs)
pubs = pubs.set_index(0).sort_values(1, ascending=False)
pubs[pubs[1]>50]

In [ ]:
for n in G.predecessors('http://www.cdc.gov/mmwr/volumes/65/ss/ss6512a1.htm'):
    print(n)

In [ ]:
TopNodes = sorted(G.in_degree, key=lambda x: x[1], reverse=True)

In [ ]:
for t in TopNodes[:40]:
    print(t[1], '\t', t[0])